In [1]:
suppressPackageStartupMessages({
    suppressWarnings({
        library(Seurat, quietly = T)
        library(openxlsx, quietly = T)
        library(ggpubr, quietly = T)
        library(plyr, quietly = T)
        library(dplyr, quietly = T)

    })
})

In [2]:
server = 'mando'
if (server == 'jabba'){
    data_path = '/data3/hratch/norcross_abc/'
}else if (server == 'mando'){
    data_path = '/data/hratch/norcross_abc/'
}

In [3]:
abc.integrated<-readRDS(paste0(data_path, 'processed/abc_annotated.RDS'))

Since we are testing differences in the same cell type across contexts, we employ DE tests that can control for technical effects. Latent variables that account for technical effects have been [shown](https://www.biorxiv.org/content/10.1101/2022.03.15.484475v1) to be effective for DE across contexts. We will use MAST and the CDR (cellular detection rate) which has been [shown](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-015-0844-5) to be an effective latent variable for technical effects

# CDR

First, we calculate the CDR from the LogNormalized expression matrix:



In [4]:
freq<-function(expr){
    nonzero.counts<-rowSums(expr !=0 ) # get # of nonzero cells per gene
    return(nonzero.counts/dim(expr)[[2]])
}

expr = abc.integrated@assays$RNA@data # log-normalized matrix
expr<-expr[which(freq(expr)>0),] # remove invariant genes

In [5]:
thresh = 0 # calculate CDR on non-zero frequency (NOTE: code will need to be changed if setting higher thresh)
if (thresh == 0){
    cdr<-unlist(unname(scale(colSums(expr!=thresh))[, 1])) # calculate CDR as in MAST tutorial (https://www.bioconductor.org/packages/release/bioc/vignettes/MAST/inst/doc/MAITAnalysis.html)
    cdr.2<-unlist(unname(colSums(expr > thresh)/dim(expr)[[1]])) # calculate as in MAST manuscript
}else{
    stop('Need to implement this if using')
}

Note, although the two methods to calculate the CDR give different absolute values, they have perfect correlation (we will proceed with the tutorial recommended CDR calculation):

In [6]:
identical(cdr, cdr.2)
cor(cdr, cdr.2,  method = "spearman", use = "complete.obs")

[1] FALSE

[1] 1

In [7]:
abc.integrated@meta.data[['cellular.detection.rate']]<-cdr # add cdr to object

# RUN MAST DE

## Part 1: Run to compare a single cell type across contexts

Specifiy the cell types and context comparisons to test for:

In [137]:
cell.types.l1<-setdiff(levels(abc.integrated$Cell.Type.Level1), 'Unknown')

# t cell subsets
md<-abc.integrated@meta.data
cell.types.l2<-levels(droplevels((md[md$Cell.Type.Level1 == 'T-cells', ][['Cell.Type.Level2']])))

comparisons<-list(c('ABC', 'UNTR'),  # second entry is baseline
                  c('DT_ABC', 'DT_Veh'), 
                  c('DT_ABC', 'ABC'))

In [112]:
# compare a cell type between contexts
MAST.de.context<-function(cell.type, context.treat, context.base, 
                          latent.vars, min.pct, lfc.thresh, 
                          cell.level, # 1 or 2
                          so, # seurat object
                          n_samples_thresh = 50 # min # of cells in the comparison
                         ){
#     abc.subset<-subset(x = so, subset = Cell.Type.Level2 == cell.type)
    cl <- FetchData(object = so, vars = paste0('Cell.Type.Level', cell.level))
    abc.subset = so[, which(x = cl == cell.type)]
    
    Idents(abc.subset)<-'orig.ident'
    
#     n_samples<-dim(abc.subset@meta.data[abc.subset@meta.data$orig.ident %in% c(context.treat, context.base),])[[1]]
#     if (n_samples<n_samples_thresh){
#         return (NULL)
    n_samples.treat<-dim(abc.subset@meta.data[abc.subset@meta.data$orig.ident == context.treat,])[[1]]
    n_samples.base<-dim(abc.subset@meta.data[abc.subset@meta.data$orig.ident == context.base,])[[1]]
    if ((n_samples.treat<n_samples_thresh) | (n_samples.base<n_samples_thresh)){
        return (NULL)
    }else{
        suppressWarnings({
            suppressMessages({
                de.res<-FindMarkers(object = abc.subset, 
                                    ident.1 = context.treat, ident.2 = context.base,
                                    assay = 'RNA', only.pos = F, 
                                    slot = 'data', test.use = 'MAST', 
                                    latent.vars = latent.vars,
                                    min.pct = min.pct, 
                                    logfc.threshold = lfc.thresh 
                                                  )
                })
        })

        names(de.res)[names(de.res) == 'p_val_adj'] <- 'bonferroni.adjusted' # rename to specify correction type
        # get the B-H to be less stringent than the native Seurat Bonferroni
        de.res[['BH.adjusted']]<-p.adjust(p = de.res$p_val, method = "BH") 
        de.res[['gene']]<-rownames(de.res)
        de.res[['Cell.Type']]<-cell.type
        de.res[['Cell.Type.Level']]<-cell.level
        de.res[['Comparison']]<-paste0(context.treat, '_vs_', context.base)

        return(de.res) 
    }
}

Filter for cell types with very few cells:

In [138]:
# total_iters = (length(comparisons)*length(cell.types.l1)) + (length(comparisons)*length(cell.types.l2))
# iter<-1
# print('begin de')
# MAST.de.res<-list()
# for (comparison in comparisons){
#     context.treat<-comparison[[1]]
#     context.base<-comparison[[2]]
#     for (ct in cell.types.l1){
#         print(paste0(iter, ' of ', total_iters))
#         cond.name<-paste0(ct, '_', paste0(comparison, collapse = 'vs'))

#         MAST.de.res[[cond.name]]<-MAST.de.context(ct, context.treat, context.base, 
#                                                   latent.vars = 'cellular.detection.rate', 
#                                                   min.pct = 0.1, lfc.thresh = 0.5,
#                                                   cell.level = '1',
#                                                   so = abc.integrated)
#         iter<-iter + 1
#         }
#     for (ct in cell.types.l2){
#         print(paste0(iter, ' of ', total_iters))
#         cond.name<-paste0(ct, '_', paste0(comparison, collapse = 'vs'))

#         MAST.de.res[[cond.name]]<-MAST.de.context(ct, context.treat, context.base, 
#                                                   latent.vars = 'cellular.detection.rate', 
#                                                   min.pct = 0.1, lfc.thresh = 0.5,
#                                                   cell.level = '2',
#                                                   so = abc.integrated)
#         iter<-iter + 1
#     }
# }
# saveRDS(MAST.de.res, paste0(data_path, 'processed/MAST_condition-specific_DE.RDS'))
# print('complete')
MAST.de.res<-readRDS(paste0(data_path, 'processed/MAST_condition-specific_DE.RDS'))

# you are here

In [139]:
de.res<-do.call("rbind", MAST.de.res)

de.res<-de.res[de.res$BH.adjusted <= 0.1,] # filter on BH
de.res<-de.res[with(de.res, order(Cell.Type, -abs(avg_log2FC), BH.adjusted)), ]

print('# of DE genes after filtering:')
table(de.res$Cell.Type, de.res$Comparison)

[1] "# of DE genes after filtering:"


                
                 ABC_vs_UNTR DT_ABC_vs_ABC DT_ABC_vs_DT_Veh
  CD4+ TE                  0           399              221
  CD4+ Treg              268           309                0
  CD4+TN                  87           151               86
  CD8+ TE/Ex               0             0              274
  CD8+ TEA_1             212           228                0
  CD8+ TEA_2             109           410                0
  CD8+ TEx prec            0           421              247
  CD8+ TN                 89           113               49
  CD8+ TN/EA-ISG          37           225                0
  cDC_1                    0           236              166
  cDC_2                    0           259               63
  gd-T                    17           192              286
  Mf                       0           598              661
  ncMono                   0             0               87
  NK  cells                0           276                0
  pDC                  

In [151]:
counter<-1
context_comparisons_workbook<-createWorkbook()
for (comparison in unique(de.res$Comparison)){
    for (cell.type in  unique(de.res$Cell.Type)){
        de.res.cl<-de.res[(de.res$Comparison == comparison) & (de.res$Cell.Type == cell.type), ]
        if (dim(de.res.cl)[[1]] > 0){
            rownames(de.res.cl)<-1:dim(de.res.cl)[[1]]
        
        addWorksheet(context_comparisons_workbook, paste0(counter))
        writeData(context_comparisons_workbook, sheet = paste0(counter), x = de.res.cl)
        counter<-counter+1
    
        write.csv(de.res.cl, 
                  paste0(data_path, 'processed/', gsub('/', '_', cell.type), '_', comparison, 'MAST_IPA_input.csv'))
        }
    }
}
saveWorkbook(context_comparisons_workbook, overwrite = T, 
                 paste0(data_path, 'processed/', 'MAST_condition-specific_DE.xlsx'))

In [152]:
paste0(data_path, 'processed/', 'MAST_condition-specific_DE.xlsx')

[1] "/data/hratch/norcross_abc/processed/MAST_condition-specific_DE.xlsx"

## Part 2: Run to compare different cell types, independent of context

Based on trajectory analysis, also compare CD8+ TEA_1 w/ CD8+ TEA_2 in the combined DT_ABC/aCD4_ABC conditions. 

In [38]:
# compare between cell types within a given context(s)
MAST.de.celltypes<-function(contexts, cell.type.treat, cell.type.base, latent.vars, min.pct, lfc.thresh, 
                           so = abc.integrated, cell.level = '2'){
    abc.subset<-subset(x = so, subset = orig.ident %in% contexts)
    Idents(abc.subset)<-paste0('Cell.Type.Level', cell.level)
    
    suppressWarnings({
        suppressMessages({
            de.res<-FindMarkers(object = abc.subset, 
                                ident.1 = cell.type.treat, ident.2 = cell.type.base,
                                assay = 'RNA', only.pos = F, 
                                slot = 'data', test.use = 'MAST', 
                                latent.vars = latent.vars,
                                min.pct = min.pct, 
                                logfc.threshold = lfc.thresh 
                                              )
            })
    })
    
    names(de.res)[names(de.res) == 'p_val_adj'] <- 'bonferroni.adjusted' # rename to specify correction type
    # get the B-H to be less stringent than the native Seurat Bonferroni
    de.res[['BH.adjusted']]<-p.adjust(p = de.res$p_val, method = "BH") 
    de.res[['gene']]<-rownames(de.res)
    de.res[['Combined.Contexts']]<-paste0(contexts, collapse=";")
    de.res[['Comparison']]<-paste0(cell.type.treat, '_vs_', cell.type.base)
    
    return(de.res)
}

In [46]:
contexts<-list(c('DT_ABC', 'aCD4_ABC'))
comparisons<-list(c('CD8+ TEA_1', 'CD8+ TEA_2'))

In [47]:
MAST.de.res.ct<-list()
for (comparison in comparisons){
    for (contexts_ in contexts){
        ct.treat<-comparison[[1]]
        ct.base<-comparison[[2]]
        cond.name<-paste0(paste0(contexts, collapse=";"), '_', paste0(comparison, collapse = 'vs'))

        MAST.de.res.ct[[cond.name]]<-MAST.de.celltypes(contexts_, ct.treat, ct.base, 
                                      latent.vars = 'cellular.detection.rate', 
                                         min.pct = 0.1, lfc.thresh = 0.5)
    }
}


In [48]:
de.res.ct<-do.call("rbind", MAST.de.res.ct)
de.res.ct<-de.res.ct[de.res.ct$BH.adjusted <= 0.1,]

print('# of DE genes after filtering:')
table(de.res.ct$Combined.Contexts, de.res.ct$Comparison)

[1] "# of DE genes after filtering:"


                 
                  CD8+ TEA_1_vs_CD8+ TEA_2
  DT_ABC;aCD4_ABC                      429

In [50]:
write.csv(de.res.ct, paste0(data_path, 'processed/', 'CD8TEA_de.csv'))